In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, callback
import dash_bootstrap_components as dbc

In [18]:
nhl_df = pd.read_excel("nhlupdates12.xlsx")
team_df = pd.read_csv("teamdata.csv")

In [3]:
#add ranking column
nhl_df["Goals Against Per Game Difference Rank"] = nhl_df.groupby(by=["Season"])["Goals Against Per Game Difference"].rank(method = "min")

In [8]:
#remove accent from montreal
nhl_df["Team"] = nhl_df["Team"].replace({"Montréal Canadiens" : "Montreal Canadiens"})

In [12]:
#merge two datasets on team names
nhl_df = pd.merge(nhl_df, team_df, how = "inner", left_on = "Team", right_on = "espn_team_name")

In [14]:
#drop unnecessary columns
nhl_df.drop(columns = ["refe_abrName", "espn_team_name", "tertiary", "quaternary"], axis = 1, inplace=True)

In [52]:
#get background color based on column value
def get_color(metric, row):
    if row[metric] >= 0:
        return row["primary"]
    elif row[metric] < 0:
        return row["primary"]
    
        
            
    

In [15]:
#get text color based on team
def text_color(row):
    if row["Team"] == "Detroit Red Wings":
        return row["primary"]
    elif row["Team"] == "Toronto Maple Leafs":
        return row["primary"]
    elif row["Team"] == "Tampa Bay Lightning":
        return row["primary"]
    elif row["Team"] == "Arizona Coyotes":
        return row["primary"]
    else:
        return "white"
    

In [16]:
#apply function
nhl_df["text_color"] = nhl_df.apply(lambda row: text_color(row), axis = 1 )

In [53]:
#apply function
nhl_df['Goals For Per Game Difference Color'] = nhl_df.apply (lambda row: get_color("Goals For Per Game Difference", row), axis=1)
nhl_df['Powerplay Percentage Difference Color'] = nhl_df.apply (lambda row: get_color("Powerplay Percentage Difference", row), axis=1)
nhl_df['Shots Per Game Difference Color'] = nhl_df.apply (lambda row: get_color("Shots Per Game Difference", row), axis=1)
nhl_df['Goals Against Per Game Difference Color'] = nhl_df.apply (lambda row: get_color("Goals Against Per Game Difference", row), axis=1)
nhl_df['Penalty Kill Percentage Difference Color'] = nhl_df.apply (lambda row: get_color("Penalty Kill Percentage Difference", row), axis=1)
nhl_df['Shots Allowed Per Game Difference Color'] = nhl_df.apply (lambda row: get_color("Shots Allowed Per Game Difference", row), axis=1)

In [46]:
nhl_df.rename(columns = {"GF/GP" : "Team Goals For Per Game", "GA/GP" : "Team Goals Against Per Game",
                         "PP%" : "Team Powerplay Percentage", "PK%" : "Team Penalty Kill Percentage",
                         "S/GP" : "Team Shots Per Game", "SA/GP" : "Team Shots Allowed Per Game"}, inplace = True)
                         

In [23]:
nhl_df.rename(columns = {"Goals For Per Game Difference" : "Goals For Per Game", "Goals Against Per Game Difference" : "Goals Against Per Game",
                         "Powerplay Percentage Difference" : "Powerplay Percentage", "Penalty Kill Percentage Difference" : "Penalty Kill Percentage",
                         "Shots Per Game Difference" : "Shots Per Game", "Shots Allowed Per Game Difference": "Shots Allowed Per Game"}, inplace = True)
                         

In [25]:
nhl_df.rename(columns = {"Goals For Per Game Difference Color" : "Goals For Per Game Color", "Goals Against Per Game Difference Color" : "Goals Against Per Game Color",
                         "Powerplay Percentage Difference Color" : "Powerplay Percentage Color", "Penalty Kill Percentage Difference Color" : "Penalty Kill Percentage Color",
                         "Shots Per Game Difference Color" : "Shots Per Game Color", "Shots Allowed Per Game Difference Color": "Shots Allowed Per Game Color"}, inplace = True)
                         

In [43]:
nhl_df.rename(columns = {"Goals For Per Game Season Avg_x" : "Goals For Per Game Season Avg"}, inplace = True)

In [52]:
external_stylesheets = [dbc.themes.BOOTSTRAP]

app = Dash(__name__, external_stylesheets = external_stylesheets)



app.layout = html.Div([

    dbc.Row([
    dbc.Col([
        html.Label('Team:'),
        dcc.Dropdown(nhl_df["Team"].sort_values().unique(),
            id='team_search',
            value="Arizona Coyotes",
            clearable=False
        ),
    ], width = 4),
    dbc.Col([
        html.Label("Metric:"),
        dcc.Dropdown(
            options=[{"label": x, "value": x} for x in nhl_df.columns[33:39]],
            id = "metric_search",
            value = "Goals For Per Game",
            clearable = False
        ),
    ], width = 4),
    dbc.Col([
        html.Label("Season:"),
        dcc.Dropdown(
            options = [{"label": x, "value" : x} for x in nhl_df["Season"].sort_values(ascending=False).unique()],
            id = "season_search",
            value = "2022-2023",
            clearable = False
        ),
    ], width = 4)]),
html.Br(),   
dbc.Row(
    [
    dbc.Col(id = 'rankings', width = 3),
    dbc.Col(id = "ga_rankings", width = 3),
    dbc.Col(id = "pp_rankings", width = 3),
    dbc.Col(id = "pk_rankings", width = 3)
    ]),
html.Br(),
dbc.Row([
    dbc.Col([dcc.Graph(id="graph")])]
                 ),
html.Br(),
dbc.Row([
    dbc.Col([dcc.Graph(id="graph2")])
])
])

    


@app.callback(
    Output("graph", "figure"),
    Input("team_search", "value"),
    Input("metric_search", "value")
)
def team_graph(team, metric):
    fig = go.Figure()
    df2 = nhl_df[nhl_df["Team"] == team]
    df2 = df2.sort_values(by="Season")
    df_color = df2[["secondary"]].reset_index()
    df_color = df_color["secondary"][0]
    text_color = df2[["text_color"]].reset_index()
    text_color = text_color["text_color"][0]
    fig.add_trace(go.Bar(x=df2["Season"], y = df2[metric], marker_color = df2[metric + " Color"], name = team, customdata = df2[[metric + " Season Avg",
                                                                                                                                "Team " + metric]]))
    fig.update_layout(title = team + " " + metric + " Vs. NHL Average By Season",
                      xaxis_title = "Season",
                      yaxis_title = "Difference from League Average",
                      title_font_size = 18,
                      paper_bgcolor = df_color,
                      plot_bgcolor='white',
                      title_font_color= text_color,
                      font_color = text_color)
    fig.update_traces(marker_line_color =  'black', marker_line_width = 1, hovertemplate = "<b>Season<b>: %{x}<br>"
                                                                                           "<b>NHL Avg: %{customdata[0]:0.3f}<br>"
                                                                                           "<b>Team Avg: %{customdata[1]:0.3f}<br>"
                                                                                           "<b>Difference<b>: %{y}<br>"
                                                                                           "<extra></extra>")
    fig.update_xaxes(showgrid = False, showticklabels = False)
    fig.update_yaxes(showgrid = False)
    return fig

@app.callback(
    Output("graph2", "figure"),
    Input("team_search", "value"),
    Input("metric_search", "value")
)

def win_pct_graph(team, metric):
    df = nhl_df[nhl_df["Team"] == team]
    df = df.sort_values(by="Season")
    df_color = df[["secondary"]].reset_index()
    df_color = df_color["secondary"][0]
    df_color2 = df[["primary"]].reset_index()
    df_color2 = df_color2["primary"][0]
    text_color = df[["text_color"]].reset_index()
    text_color = text_color["text_color"][0]
    fig = px.scatter(df, x = df["Points Percentage"], y = df[metric], title = team + " " + metric + " Vs. Win Percentage By Season",
                     custom_data= df[['Season', metric + " Season Avg", "Team " + metric]])
    fig.update_layout(title_font_size = 18,
                     yaxis_title = metric + " Difference from League Average",
                     xaxis_title = "Win Percentage",
                     paper_bgcolor = df_color,
                     plot_bgcolor='white',
                     title_font_color= text_color,
                     font_color = text_color)
    fig.update_traces(marker = dict(color = df_color2, size = 15), marker_line_color =  'black', marker_line_width = 1,
                     hovertemplate = "<b>Season<b>: %{customdata[0]}<br>" 
                                     "<b>Win Pct: %{x:0.3f}<br>" 
                                     "<b>NHL Avg: %{customdata[1]:0.3f}<br>" 
                                     "<b>Team Avg: %{customdata[2]:0.3f}<br>" 
                                     "<b>Difference<b>: %{y}<br>" 
                                     "<extra></extra>")
    fig.update_xaxes(showgrid = False)
    fig.update_yaxes(showgrid = False)
    return fig

def write_text(input_text):

    return dbc.Card(
                dbc.CardBody([html.H5(input_text)], style={'textAlign': 'center'}), color = "primary", inverse = True)

@app.callback(
    Output("rankings", "children"),
    Input("team_search", "value"),
    Input("season_search","value"))
def gf_rank(team, season):
    data = nhl_df[nhl_df["Team"] == team]
    data = data[data["Season"] == season]
    try:
        card1 = data["Goals For Per Game Difference Rank"].reset_index()
        card1 = card1["Goals For Per Game Difference Rank"][0]
    except:
        card1 = "N/A"
    card_content = [
    dbc.CardBody([html.H5(season + " Goals For Rank: " + str(card1))], style={'textAlign': 'center'})]

  
    return  dbc.Card(card_content, color = data["primary"], inverse=True)

@app.callback(
    Output("ga_rankings", "children"),
    Input("team_search", "value"),
    Input("season_search","value"))
def ga_rank(team, season):
    data = nhl_df[nhl_df["Team"] == team]
    data = data[data["Season"] == season]
    try:
        card1 = data["Goals Against Per Game Difference Rank"].reset_index()
        card1 = card1["Goals Against Per Game Difference Rank"][0].astype(int)
    except:
        card1 = "N/A"
    card_content = [
    dbc.CardBody([html.H5(season + " Goals Against Rank: " + str(card1))], style={'textAlign': 'center'})]
        
    return  dbc.Card(card_content, color = data["primary"], inverse=True)
        
@app.callback(
    Output("pp_rankings", "children"),
    Input("team_search", "value"),
    Input("season_search","value"))
def pp_rank(team, season):
    data = nhl_df[nhl_df["Team"] == team]
    data = data[data["Season"] == season]
    try:
        card1 = data["Powerplay Percentage Difference Rank"].reset_index()
        card1 = card1["Powerplay Percentage Difference Rank"][0].astype(int)
    except:
        card1 = "N/A"
    card_content = [
    dbc.CardBody([html.H5(season + " Powerplay Rank: " + str(card1))], style={'textAlign': 'center'})]

  #if else for return statement; if color is white
    return  dbc.Card(card_content, color = data["primary"], inverse=True)

@app.callback(
    Output("pk_rankings", "children"),
    Input("team_search", "value"),
    Input("season_search","value"))
def pp_rank(team, season):
    data = nhl_df[nhl_df["Team"] == team]
    data = data[data["Season"] == season]
    try:
        card1 = data["Penalty Kill Percentage Difference Rank"].reset_index()
        card1 = card1["Penalty Kill Percentage Difference Rank"][0].astype(int)
    except:
        card1 = "N/A"
    card_content = [
    dbc.CardBody([html.H5(season + " Penalty Kill Rank: " + str(card1))], style={'textAlign': 'center'})]

  
    return  dbc.Card(card_content, color = data["primary"], inverse=True)
    
    

if __name__ == "__main__":
    app.run()
        